In [1]:
#Run this twice!
!git clone https://github.com/Haydenlee2024/altdata.git
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader
!pip install stockstats
!pip install finance-datareader
from stockstats import StockDataFrame
import pandas_datareader as pdr
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib
import numpy as np
from datetime import datetime, timedelta
import FinanceDataReader as fdr
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt


fatal: destination path 'altdata' already exists and is not an empty directory.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3805    0  3805    0     0  20026      0 --:--:-- --:--:-- --:--:-- 20026
100  503k  100  503k    0     0   923k      0 --:--:-- --:--:-- --:--:--  923k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3813    0  3813    0     0  19160      0 --:--:-- --:--:-- --:--:-- 19065
100  406k  100  406k    0     0   759k      0 --:--:-- --:--:-- --:--:--  759k


In [2]:
!pip install nasdaq-data-link
import nasdaqdatalink

KOREA

In [2]:
tickers = pd.read_csv('/content/altdata/tickers_kr.csv')
tickers = tickers[['종목코드', '종목명', '기초지수_지수명']]
tickers_kospi200 = tickers[tickers['기초지수_지수명'] == '코스피 200'].reset_index(drop = True)
tickers_kospi100 = tickers[tickers['기초지수_지수명'] == '코스피 100'].reset_index(drop = True)
tickers_kospi = tickers[tickers['기초지수_지수명'] == '코스피'].reset_index(drop = True)
tickers_kosdaq150 = tickers[tickers['기초지수_지수명'] == '코스닥 150'].reset_index(drop = True)
tickers_snp = tickers[tickers['기초지수_지수명'] == 'S&P 500'].reset_index(drop = True)
tickers_nasdaq100 = tickers[tickers['기초지수_지수명'] == 'NASDAQ 100'].reset_index(drop = True)
tickers_krx300 = tickers[tickers['기초지수_지수명'] == 'KRX 300'].reset_index(drop = True)

In [3]:
bh = {}
strategy = {}
classifiers = {}
current = {}

In [4]:
def set_krX(ticker):
  ec = pd.read_csv('/content/altdata/DEXKOUS.csv')
  a = []
  for i in ec.index:
    if ec.loc[i]['DEXKOUS'] == '.':
      a.append(i)
  ec = ec.drop(ec.index[a])
  ec['DATE'] = pd.to_datetime(ec['DATE'])
  ec['DEXKOUS'] = ec['DEXKOUS'].astype(float)
  ec['DIFFec'] = (ec['DEXKOUS'].shift(261) - ec['DEXKOUS'])/ec['DEXKOUS']
  ec = ec.dropna()
  ec = ec.set_index('DATE', drop = True)
  ec2 = ec
  export = pd.read_csv('/content/altdata/KOREXPORTQDSMEI.csv')
  export['DIFFex'] = (export['KOREXPORTQDSMEI'].shift(4) - export['KOREXPORTQDSMEI'])/export['KOREXPORTQDSMEI']
  export = export.dropna()
  export = export[227:]
  export['DATE'] = pd.to_datetime(export['DATE'])
  export = export.set_index('DATE', drop = True)
  export2 = pd.DataFrame(columns = export.columns)
  for idx in range(len(export.index) - 1):
    diff = export.index[idx + 1] - export.index[idx]
    for i in range(1, diff.days - 1):
      if export.index[idx] + timedelta(days = i) in ec.index:
        export2.loc[export.index[idx] + timedelta(days = i)] = export.loc[export.index[idx]]
  ec = ec.drop([i for i in ec.index if i not in export2.index])
  cpi = pd.read_csv('/content/altdata/KORCPIALLMINMEI.csv')
  cpi['DIFFcpi'] = (cpi['KORCPIALLMINMEI'].shift(12) - cpi['KORCPIALLMINMEI'])/cpi['KORCPIALLMINMEI']
  cpi['DATE'] = pd.to_datetime(cpi['DATE'])
  cpi = cpi.set_index('DATE', drop = True)
  cpi = cpi.dropna()
  cpi2 = pd.DataFrame(columns = cpi.columns)
  for idx in range(len(cpi.index) - 1):
    diff = cpi.index[idx + 1] - cpi.index[idx]
    for i in range(1, diff.days - 1):
      if cpi.index[idx] + timedelta(days = i) in export2.index:
        cpi2.loc[cpi.index[idx] + timedelta(days = i)] = cpi.loc[cpi.index[idx]]
  ec = ec.drop([i for i in ec.index if i not in cpi2.index])
  export2 = export2.drop([i for i in export2.index if i not in cpi2.index])
  gdpkr = pd.read_csv('/content/altdata/GDPKR.csv')
  gdpkr['DIFFgdp'] = (gdpkr['GDP'].shift(12) - gdpkr['GDP'])/gdpkr['GDP']
  gdpkr['DATE'] = pd.to_datetime(gdpkr['DATE'])
  gdpkr = gdpkr.set_index('DATE', drop = True)
  gdpkr = gdpkr.dropna()
  gdpkr2 = pd.DataFrame(columns = gdpkr.columns)
  for idx in range(len(gdpkr.index) - 1):
    diff = gdpkr.index[idx + 1] - gdpkr.index[idx]
    for i in range(1, diff.days - 1):
      if gdpkr.index[idx] + timedelta(days = i) in export2.index:
        gdpkr2.loc[gdpkr.index[idx] + timedelta(days = i)] = gdpkr.loc[gdpkr.index[idx]]
  unrate = pd.read_csv('/content/altdata/UNRATEKR.csv')
  unrate['DIFFunrate'] = (unrate['UNRATE'].shift(12) - unrate['UNRATE'])/unrate['UNRATE']
  unrate['DATE'] = pd.to_datetime(unrate['DATE'])
  unrate = unrate.set_index('DATE', drop = True)
  unrate = unrate.dropna()
  unrate2 = pd.DataFrame(columns = unrate.columns)
  for idx in range(len(unrate.index) - 1):
    diff = unrate.index[idx + 1] - unrate.index[idx]
    for i in range(1, diff.days - 1):
      if unrate.index[idx] + timedelta(days = i) in gdpkr2.index:
        unrate2.loc[unrate.index[idx] + timedelta(days = i)] = unrate.loc[unrate.index[idx]]
  ec = ec.drop([i for i in ec.index if i not in gdpkr2.index])
  export2 = export2.drop([i for i in export2.index if i not in gdpkr2.index])
  cpi2 = cpi2.drop([i for i in cpi2.index if i not in gdpkr2.index])
  unrate2 = unrate2.drop([i for i in unrate2.index if i not in gdpkr2.index])
  total = ec.join(export2).join(cpi2).join(gdpkr2).join(unrate2)
  total.dropna()
  X = total[['DIFFec', 'DIFFex', 'DIFFcpi', 'DIFFgdp', 'DIFFunrate']]
  X = X.dropna()
  X = X.drop([i for i in X.index if i not in ticker.index])
  return [X, ec2, export, cpi, gdpkr, unrate]

In [5]:
def test_KR(ticker_name):
  
  ticker = StockDataFrame.retype(pdr.DataReader(ticker_name, data_source = 'yahoo')).reset_index()
  ticker['rsi_14']
  ticker['close_20_ema']
  ticker['close_50_ema']
  ticker['boll']
  ticker['boll_ub']
  ticker['boll_lb']
  ticker['DIFF'] = (ticker['close'].shift(-30) - ticker['close'])/ticker['close']
  ticker = ticker.dropna()
  ticker['Date'] = pd.to_datetime(ticker['Date'])
  ticker = ticker.set_index('Date', drop = True)
  Xs = set_krX(ticker)
  X = Xs[0]
  Y = []
  for i in ticker.index:
    if i in X.index:
      if ticker.loc[i]['DIFF'] > 0:
        Y.append(True)
      else:
        Y.append(False)
  xtrain,xtest,ytrain,ytest = train_test_split(X, Y, test_size=0.3, shuffle = False)
  xrtrain, xrtest, yrtrain, yrtest = train_test_split(xtrain, ytrain, test_size=2/7, shuffle = False)
  print(len(ytest))
  clf = tree.DecisionTreeClassifier(max_depth = 5, criterion = 'gini')
  clf = clf.fit(xrtrain,yrtrain)
  if clf.score(xtest, ytest) < 0.5:
    return
  #fig = plt.figure(figsize=(25,20)) 
  #tree.plot_tree(clf, feature_names = X.columns, class_names=['Buy', 'Sell'])
  money = 1000000
  etf_val = 0
  prev = xtest.index[0]
  not_trade = 0
  leverage = 0
  leverages = []
  for i in xtest.index:
    leverages.append(leverage)
    etf_val *= 1 + ((ticker.loc[i]['close'] - ticker.loc[prev]['close'])/ticker.loc[prev]['close'])*leverage
    prev = i
    if (clf.predict([[X.loc[i]['DIFFec'], X.loc[i]['DIFFex'], X.loc[i]['DIFFcpi'], X.loc[i]['DIFFgdp'], X.loc[i]['DIFFunrate']]])[0] or ticker.loc[i]['close'] < ticker.loc[i]['boll_lb'] or ticker.loc[i]['close_20_ema'] > ticker.loc[i]['close_50_ema']) and not_trade == 0:
      etf_val += money
      money = 0
      not_trade = 31
      """
      if clf.predict_proba([[X.loc[i]['DIFFec'], X.loc[i]['DIFFex'], X.loc[i]['DIFFcpi'], X.loc[i]['DIFFgdp'], X.loc[i]['DIFFunrate']]])[0][1] > 0.9:
        #print(clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][0])
        leverage = 3
      elif clf.predict_proba([[X.loc[i]['DIFFec'], X.loc[i]['DIFFex'], X.loc[i]['DIFFcpi'], X.loc[i]['DIFFgdp'], X.loc[i]['DIFFunrate']]])[0][1] > 0.5:
        #print(clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][0])
        leverage = 2
      else:
        #print(clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][0])
        leverage = 1
      """
      leverage = 1
    elif not_trade == 0 and ticker.loc[i]['rsi_14'] > 70:
      money += etf_val
      etf_val = 0
      leverage = 0
    if not_trade > 0:
      not_trade -= 1
  bh[ticker_name] = round(1000000 + 1000000 * (ticker.loc[xtest.index[-1]]['close'] - ticker.loc[xtest.index[0]]['close'])/ticker.loc[xtest.index[0]]['close'])
  strategy[ticker_name] = round(money + etf_val)
  classifiers[ticker_name] = clf
  i = X.index[-1]
  current[ticker_name] = (clf.predict([[X.loc[i]['DIFFec'], X.loc[i]['DIFFex'], X.loc[i]['DIFFcpi'], X.loc[i]['DIFFgdp'], X.loc[i]['DIFFunrate']]])[0] or ticker.loc[i]['close'] < ticker.loc[i]['boll_lb'] or ticker.loc[i]['close_20_ema'] > ticker.loc[i]['close_50_ema'])
  print(len([i for i in ytest if i])/len(ytest))
  print("Score: ",clf.score(xrtest,yrtest))
  print("Recall Score: ",recall_score(yrtest,clf.predict(xrtest)))
  print("Precision Score: ",precision_score(yrtest,clf.predict(xrtest)))
  print("F1 Score: ",f1_score(yrtest,clf.predict(xrtest))) 
  print("Score: ",clf.score(xtest,ytest))
  print("Recall Score: ",recall_score(ytest,clf.predict(xtest)))
  print("Precision Score: ",precision_score(ytest,clf.predict(xtest)))
  print("F1 Score: ",f1_score(ytest,clf.predict(xtest))) 

In [6]:
ticker_list = [tickers_kospi100, tickers_kospi200, tickers_kospi, tickers_kosdaq150, tickers_krx300]
for ct in ticker_list:
  current_ticker = ct
  for ticker in range(len(current_ticker)):
    print('%d/%d done' % (ticker, len(current_ticker)))
    try:
      test_KR(str(current_ticker.loc[ticker]['종목코드']) + '.KS')
    except:
      print(current_ticker.loc[ticker]['종목코드'], "Doesn't work!")


0/4 done
233
1/4 done
233
2/4 done
233


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

0.7339055793991416
Score:  0.4358974358974359
Recall Score:  0.5089285714285714
Precision Score:  0.6333333333333333
F1 Score:  0.5643564356435643
Score:  0.5407725321888412
Recall Score:  0.5964912280701754
Precision Score:  0.7285714285714285
F1 Score:  0.6559485530546623
3/4 done
233
0/14 done
233
1/14 done
218
2/14 done
391670 Doesn't work!
3/14 done
391680 Doesn't work!
4/14 done
233
5/14 done
233
6/14 done
233
7/14 done
69500 Doesn't work!
8/14 done
233
9/14 done
69660 Doesn't work!
10/14 done
233
11/14 done
233
12/14 done
233
13/14 done
233
0/10 done
233
1/10 done
402520 Doesn't work!
2/10 done
196
3/10 done
183
4/10 done
233
5/10 done
364690 Doesn't work!
6/10 done
365040 Doesn't work!
7/10 done
233
8/10 done
385720 Doesn't work!
9/10 done
403790 Doesn't work!
0/9 done
7
0.0
Score:  0.8
Recall Score:  0.5
Precision Score:  1.0
F1 Score:  0.6666666666666666
Score:  1.0
Recall Score:  0.0
Precision Score:  0.0
F1 Score:  0.0
1/9 done


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

192
2/9 done
233
3/9 done
354500 Doesn't work!
4/9 done
233
5/9 done
233


...

0.5965665236051502
Score:  0.6538461538461539
Recall Score:  0.625
Precision Score:  0.7692307692307693
F1 Score:  0.6896551724137931
Score:  0.5064377682403434
Recall Score:  0.2158273381294964
Precision Score:  0.8333333333333334
F1 Score:  0.3428571428571428
6/9 done
164
7/9 done
233


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

0.6008583690987125
Score:  0.4358974358974359
Recall Score:  0.3269230769230769
Precision Score:  0.6538461538461539
F1 Score:  0.4358974358974359
Score:  0.5278969957081545
Recall Score:  0.21428571428571427
Precision Score:  1.0
F1 Score:  0.35294117647058826
8/9 done
233


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

0.592274678111588
Score:  0.26282051282051283
Recall Score:  0.2708333333333333
Precision Score:  0.36619718309859156
F1 Score:  0.31137724550898205
Score:  0.5622317596566524
Recall Score:  0.26811594202898553
Precision Score:  0.9736842105263158
F1 Score:  0.42045454545454547
0/9 done
219
1/9 done
219
2/9 done
192
3/9 done
219
4/9 done
182
5/9 done
219
6/9 done
182
7/9 done
219
8/9 done
219


In [7]:
print(bh)
print(strategy)
print(current)

{'159800.KS': 1472284, '301400.KS': 974329, '233740.KS': 1405317, '232080.KS': 1226036, '233160.KS': 1405536}
{'159800.KS': 1472284, '301400.KS': 1000000, '233740.KS': 1405317, '232080.KS': 1226036, '233160.KS': 1405536}
{'159800.KS': True, '301400.KS': False, '233740.KS': False, '232080.KS': False, '233160.KS': True}


US (PMI, Unemployment rate)

In [12]:
def set_usX(ticker):
  usgdp = pd.read_csv('/content/altdata/GDP.csv')
  usgdp['DIFFgdp'] = (usgdp['GDP'].shift(4) - usgdp['GDP'])/usgdp['GDP']
  usgdp['DATE'] = pd.to_datetime(usgdp['DATE'])
  usgdp = usgdp.set_index('DATE', drop = True)
  usgdp = usgdp.dropna()
  usgdp2 = pd.DataFrame(columns = usgdp.columns)
  for idx in range(len(usgdp.index) - 1):
    diff = usgdp.index[idx + 1] - usgdp.index[idx]
    for i in range(1, diff.days - 1):
      if usgdp.index[idx] + timedelta(days = i) in ticker.index:
        usgdp2.loc[usgdp.index[idx] + timedelta(days = i)] = usgdp.loc[usgdp.index[idx]]
  uspmi = pd.read_csv('/content/altdata/ISM-MAN_PMI.csv')
  uspmi['DIFFpmi'] = (uspmi['PMI'].shift(12) - uspmi['PMI'])/uspmi['PMI']
  uspmi['Date'] = pd.to_datetime(uspmi['Date'])
  uspmi = uspmi.sort_values('Date')
  uspmi = uspmi.set_index('Date', drop = True)
  uspmi = uspmi.dropna()
  uspmi2 = pd.DataFrame(columns = uspmi.columns)
  for idx in range(len(uspmi.index) - 1):
    diff = uspmi.index[idx + 1] - uspmi.index[idx]
    for i in range(1, diff.days - 1):
      if uspmi.index[idx] + timedelta(days = i) in usgdp2.index:
        uspmi2.loc[uspmi.index[idx] + timedelta(days = i)] = uspmi.loc[uspmi.index[idx]]
  usunrate = pd.read_csv('/content/altdata/UNRATE.csv')
  usunrate['DIFFunrate'] = (usunrate['UNRATE'].shift(12) - usunrate['UNRATE'])/usunrate['UNRATE']
  usunrate['DATE'] = pd.to_datetime(usunrate['DATE'])
  usunrate = usunrate.set_index('DATE', drop = True)
  usunrate = usunrate.dropna()
  usunrate2 = pd.DataFrame(columns = usunrate.columns)
  for idx in range(len(usunrate.index) - 1):
    diff = usunrate.index[idx + 1] - usunrate.index[idx]
    for i in range(1, diff.days - 1):
      if usunrate.index[idx] + timedelta(days = i) in usgdp2.index:
        usunrate2.loc[usunrate.index[idx] + timedelta(days = i)] = usunrate.loc[usunrate.index[idx]]
  usgdpdef = pd.read_csv('/content/altdata/GDPDEF.csv')
  usgdpdef['DIFFgdpdef'] = (usgdpdef['GDPDEF'].shift(4) - usgdpdef['GDPDEF'])/usgdpdef['GDPDEF']
  usgdpdef['DATE'] = pd.to_datetime(usgdpdef['DATE'])
  usgdpdef = usgdpdef.set_index('DATE', drop = True)
  usgdpdef = usgdpdef.dropna()
  usgdpdef2 = pd.DataFrame(columns = usgdpdef.columns)
  for idx in range(len(usgdpdef.index) - 1):
    diff = usgdpdef.index[idx + 1] - usgdpdef.index[idx]
    for i in range(1, diff.days - 1):
      if usgdpdef.index[idx] + timedelta(days = i) in usgdp2.index:
        usgdpdef2.loc[usgdpdef.index[idx] + timedelta(days = i)] = usgdpdef.loc[usgdpdef.index[idx]]
  total = usgdp2.join(usgdpdef2).join(usunrate2).join(uspmi2)
  total.dropna()
  X = total[['DIFFgdp', 'DIFFgdpdef', 'DIFFunrate', 'DIFFpmi']]
  X = X.dropna()
  X = X.drop([i for i in X.index if i not in ticker.index])
  return [X, usgdp, usgdpdef, usunrate, uspmi]

In [28]:
bh = {}
strategy = {}
strategy_original = {}
classifiers = {}
current = {}
current_original = {}

In [33]:
def test_US(ticker_name):
  
  ticker = StockDataFrame.retype(pdr.DataReader(ticker_name, data_source = 'yahoo')).reset_index()
  ticker['rsi_14']
  ticker['close_20_ema']
  ticker['close_50_ema']
  ticker['boll']
  ticker['boll_ub']
  ticker['boll_lb']
  ticker['DIFF'] = (ticker['close'].shift(-30) - ticker['close'])/ticker['close']
  ticker = ticker.dropna()
  ticker['Date'] = pd.to_datetime(ticker['Date'])
  ticker = ticker.set_index('Date', drop = True)
  Xs = set_usX(ticker)
  X = Xs[0]
  Y = []
  for i in ticker.index:
    if i in X.index:
      if ticker.loc[i]['DIFF'] > 0.04:
        Y.append(True)
      else:
        Y.append(False)
  xtrain,xtest,ytrain,ytest = train_test_split(X, Y, test_size=0.3, shuffle = False)
  xrtrain, xrtest, yrtrain, yrtest = train_test_split(xtrain, ytrain, test_size=2/7, shuffle = False)
  clf = tree.DecisionTreeClassifier(max_depth = 5, criterion = 'gini')
  clf = clf.fit(xrtrain,yrtrain)
  if clf.score(xtest, ytest) < 0.5:
    return
  #fig = plt.figure(figsize=(25,20)) 
  #tree.plot_tree(clf, feature_names = X.columns, class_names=['Buy', 'Sell'])
  money = 1000000
  money_original = 1000000
  etf_val = 0
  etf_val_original = 0
  prev = xtest.index[0]
  not_trade = 0
  not_trade_original = 0
  for i in xtest.index:
    etf_val *= 1 + ((ticker.loc[i]['close'] - ticker.loc[prev]['close'])/ticker.loc[prev]['close'])
    etf_val_original *= 1 + ((ticker.loc[i]['close'] - ticker.loc[prev]['close'])/ticker.loc[prev]['close'])
    prev = i
    if clf.predict([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0]:
      etf_val_original += money_original
      money_original = 0
      not_trade_original = 30
    if clf.predict([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0] or ticker.loc[i]['close'] < ticker.loc[i]['boll_lb'] or ticker.loc[i]['close_20_ema'] > ticker.loc[i]['close_50_ema'] and not_trade == 0:
      etf_val += money
      money = 0
      not_trade = 30
      #if clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][1] > 0.9:
        #print(clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][0])
        #leverage = 3
      #elif clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][1] > 0.5:
        #print(clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][0])
        #leverage = 2
      #else:
        #print(clf.predict_proba([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0][0])
        #leverage = 1
    elif not_trade == 0:
      money += etf_val
      etf_val = 0
    elif not_trade_original == 0 :
      money_original += etf_val_original
      etf_val_original = 0
    if not_trade > 0:
      not_trade -= 1
    if not_trade_original > 0:
      not_trade_original -= 1
  bh[ticker_name] = 1000000 + 1000000 * (ticker.loc[xtest.index[-1]]['close'] - ticker.loc[xtest.index[0]]['close'])/ticker.loc[xtest.index[0]]['close']
  strategy[ticker_name] = money + etf_val
  strategy_original[ticker_name] = money_original + etf_val_original
  classifiers[ticker_name] = clf
  current[ticker_name] = clf.predict([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0] or ticker.loc[i]['close'] < ticker.loc[i]['boll_lb'] or ticker.loc[i]['close_20_ema'] > ticker.loc[i]['close_50_ema']
  current_original[ticker_name] = clf.predict([[X.loc[i]['DIFFgdp'], X.loc[i]['DIFFgdpdef'], X.loc[i]['DIFFunrate'], X.loc[i]['DIFFpmi']]])[0]
  #fig = plt.figure(figsize=(25,20)) 
  #tree.plot_tree(clf, feature_names = X.columns, class_names=['Buy', 'Sell'])
  print("Score: ",clf.score(xrtest,yrtest))
  print("Recall Score: ",recall_score(yrtest,clf.predict(xrtest)))
  print("Precision Score: ",precision_score(yrtest,clf.predict(xrtest)))
  print("F1 Score: ",f1_score(yrtest,clf.predict(xrtest))) 
  print("Score: ",clf.score(xtest,ytest))
  print("Recall Score: ",recall_score(ytest,clf.predict(xtest)))
  print("Precision Score: ",precision_score(ytest,clf.predict(xtest)))
  print("F1 Score: ",f1_score(ytest,clf.predict(xtest))) 

In [24]:
ticker = StockDataFrame.retype(pdr.DataReader('133690.KS', data_source = 'yahoo')).reset_index()
ticker['DIFF'] = (ticker['close'].shift(-30) - ticker['close'])/ticker['close']
ticker = ticker.dropna()
ticker['Date'] = pd.to_datetime(ticker['Date'])
ticker = ticker.set_index('Date', drop = True)
Xs = set_usX(ticker)
X = Xs[0]
Y = []
for i in ticker.index:
  if i in X.index:
    if ticker.loc[i]['DIFF'] > 0.04:
      Y.append(True)
    else:
      Y.append(False)

In [34]:
ticker_list = [tickers_snp, tickers_nasdaq100]
for ticker in ticker_list:
  current_tickers = ticker
  for ticker in current_tickers.index:
    print('%d/%d done' % (ticker, len(current_tickers)))
    try:
      test_US(str(current_tickers.loc[ticker]['종목코드']) + '.KS')
    except ValueError:
      continue

0/6 done


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

Score:  0.7204968944099379
Recall Score:  0.0
Precision Score:  0.0
F1 Score:  0.0
Score:  0.5991735537190083
Recall Score:  0.7364341085271318
Precision Score:  0.6012658227848101
F1 Score:  0.6620209059233448
1/6 done
2/6 done
3/6 done
4/6 done


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

Score:  0.7159090909090909
Recall Score:  0.7227722772277227
Precision Score:  0.7684210526315789
F1 Score:  0.7448979591836735
Score:  0.5735849056603773
Recall Score:  0.6203208556149733
Precision Score:  0.7341772151898734
F1 Score:  0.672463768115942
5/6 done
0/7 done
1/7 done
2/7 done
3/7 done


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with

Score:  0.5980392156862745
Recall Score:  1.0
Precision Score:  0.5980392156862745
F1 Score:  0.7484662576687116
Score:  0.7320261437908496
Recall Score:  1.0
Precision Score:  0.7320261437908496
F1 Score:  0.8452830188679245
4/7 done
5/7 done
6/7 done


In [35]:
print(bh)
print(strategy)
print(strategy_original)
print(current)
print(current_original)

{'269540.KS': 1053093.8123752496, '225040.KS': 1055651.176133104, '304940.KS': 1283832.8792007267}
{'269540.KS': 1053093.8123752482, '225040.KS': 1055651.176133104, '304940.KS': 1283832.8792007277}
{'269540.KS': 1056467.7613135756, '225040.KS': 1095238.0952380956, '304940.KS': 1283832.8792007277}
{'269540.KS': True, '225040.KS': True, '304940.KS': True}
{'269540.KS': True, '225040.KS': True, '304940.KS': True}
